In [ ]:
import re
import pandas as pd
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
from tqdm.auto import tqdm

In [ ]:
gaz = pd.read_excel('gazeta_articles.xlsx') # Газета.ру
iz = pd.read_excel('iz_articles.xlsx') # Известия
mk = pd.read_excel('mk_articles.xlsx') # Московский комсомолец
ng = pd.read_excel('ng_articles.xlsx') # Независимая газета
rg = pd.read_excel('rg_articles.xlsx') # Российская газета
rbk = pd.read_excel('rbk_articles.xlsx') # РБК

In [ ]:
# Выставляем метки изданий (даны по условию задачи)
loyal = gaz.append(rg).reset_index(drop=True)
independent = rbk.append(ng).reset_index(drop=True)
mass = iz.append(mk).reset_index(drop=True)

loyal['type'] = 'loyal'
independent['type'] = 'independent'
mass['type'] = 'mass'

In [ ]:
data = loyal.append(independent).append(mass).reset_index(drop=True)

In [ ]:
stop = stopwords.words('russian')

In [ ]:
mystem = Mystem()

def preprocess_text(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    text = url.sub(r'', text)
    
    html = re.compile(r'<.*?>')
    text = html.sub(r'',text)
    
    html = re.compile(r'{.*?}')
    text = html.sub(r'',text)
    
    eng = re.compile(r'[a-zA-Z]')
    text = eng.sub(r'', text)
    
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if (token not in stop) and (token != " ") and (token.strip() not in punctuation)]
    text = " ".join(tokens)
    text = text.replace('_', '').replace('«', '').replace('»', '').replace('.', '').replace('(', '').replace(')', '')
    text = text.replace('80', '').replace('1260', '').replace('}', '').replace(';', '').replace(',', '').replace(':', '')
    text = text.replace("'", '').replace('-', '').replace('→', '').replace('#', '').replace('3  3', '').replace('  ', '')
    text = text.replace('"', '').strip()
    
    return text

In [ ]:
tqdm.pandas()

In [ ]:
data['content'] = data['content'].progress_apply(lambda x: preprocess_text(x))

In [ ]:
data.to_csv('data.csv', index=False)